In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/2025 HOOP SHEET.csv")

In [3]:
columns = df.columns.tolist()
for i in range(len(columns)):
    temp = columns[i].strip().replace("\n", " ").split()
    columns[i] = " ".join(temp)
    print(columns[i])

NAME
30 SEC HEATING (F) (INTERNAL)
BOUNCE HEIGHT AVERAGE (cm)
HEEL HEIGHT (cm)
HEEL-BALL DROP (cm)
WEIGHT oz/g (US MEN'S 11)
DEGREE OF SLIP
SHANK SCORE
SPEED RATIO
DUROMETER
LAST SHAPE (DEGREE OF INFLARE)
LENGTH (cm)
METATARSAL WIDTH(cm)
TOEBOX WIDTH(cm)
TOE BOX TAPER (DEGREES)
RESTING POSITION (DEGREES OF EVERSION)
LOADED POSITION (DEGREES OF EVERSION)
HEEL WIDTH (cm)
NARROWEST MIDFOOT MEASUREMENT (cm)
URS
MIDSOLE FOAM
ARCH TYPES F = FLAT N = NEUTRAL H = HIGH
SHANK MATERIAL
UPPER DURABILITY 0 = NOT THROUGH 1ST LAYER 1 = THROUGH 1ST LAYER
OUTSOLE DURABILITY
BEST SURFACE
CONTAINMENT
BOUNCE
SA
SPEED
DURABILITY
COMFORT
SUPPORT
PLAYABILITY
FITS ORTHOTICS


In [4]:
df.columns = columns

In [5]:
df.dtypes

NAME                                                                 object
30 SEC HEATING (F) (INTERNAL)                                        object
BOUNCE HEIGHT AVERAGE (cm)                                          float64
HEEL HEIGHT (cm)                                                     object
HEEL-BALL DROP (cm)                                                  object
WEIGHT oz/g (US MEN'S 11)                                            object
DEGREE OF SLIP                                                       object
SHANK SCORE                                                         float64
SPEED RATIO                                                          object
DUROMETER                                                            object
LAST SHAPE (DEGREE OF INFLARE)                                       object
LENGTH (cm)                                                         float64
METATARSAL WIDTH(cm)                                                float64
TOEBOX WIDTH

In [6]:
df.loc[df["30 SEC HEATING (F) (INTERNAL)"] == "99.7/109.8", "30 SEC HEATING (F) (INTERNAL)"] = "109.8"
df['30 SEC HEATING (F) (INTERNAL)'] = df['30 SEC HEATING (F) (INTERNAL)'].astype(float)
   

In [7]:
def convert_to_float(value):
    if isinstance(value, str):
        value = value.split("(")[0].strip()
    return float(value)

df['HEEL HEIGHT (cm)'] = df['HEEL HEIGHT (cm)'].apply(convert_to_float)

In [8]:
df['BOUNCE HEIGHT AVERAGE (cm)'] = df['BOUNCE HEIGHT AVERAGE (cm)'].astype(float)

In [9]:
df['HEEL-BALL DROP (cm)'] = df['HEEL-BALL DROP (cm)'].apply(convert_to_float)

In [10]:
def convert_weight(value):
    if isinstance(value, str):
        if "OZ" or "oz" in value:
            value = value.lower().replace("oz", "").strip()
            value = value.split("/")[0].strip()
        return float(value)
    return value

df["WEIGHT oz/g (US MEN'S 11)"] = df["WEIGHT oz/g (US MEN'S 11)"].apply(convert_weight)
# change column name
df = df.rename(columns={"WEIGHT oz/g (US MEN'S 11)": "WEIGHT oz (US MEN'S 11)"})


In [11]:
df.loc[df["DEGREE OF SLIP"] == '42 dyed 38 clear ', "DEGREE OF SLIP"] = 40
df['DEGREE OF SLIP'] = df['DEGREE OF SLIP'].astype(float) 

In [12]:
value_list = df[df["SPEED RATIO"] == '2.41/2.18'].to_numpy()
value_list

array([['XERO X1 MID/HIGH', 98.7, 31.0, 1.2, 0.0, 12.85, 37.0, 0.0,
        '2.41/2.18', '16.5', '13', 27.8, 9.7, 11.0, 21.0, 2.0, 8.0, nan,
        nan, 25.0, 'EVA+POLYURETHANE', 'F,N,H', 'NONE', 1.0, '1mm',
        'HARDWOOD', 3.5, 2.0, 3.0, 3.0, 3.5, 3.5, 3.0, 3.5, 'N']],
      dtype=object)

In [13]:
df.loc[df["SPEED RATIO"] == '2.41/2.18', ["NAME", "SPEED RATIO"]] = ["XERO X1 MID", 2.41]

In [14]:
df.loc[len(df)] = value_list[0]
df.loc[len(df) - 1, ["NAME", "SPEED RATIO"]] = ["XERO X1 HIGH", 2.18]
df["SPEED RATIO"] = df["SPEED RATIO"].astype(float)

In [15]:
records = []
for i in range(len(df)):
    try:
        float(df.loc[i, "DUROMETER"])
    except ValueError:
        records.append(df.loc[i])



In [16]:
df['FLAT ARCH'] = False
df['NEUTRAL ARCH'] = False
df['HIGH ARCH'] = False
for i in range(len(df)):
    val = df.loc[i, "ARCH TYPES F = FLAT N = NEUTRAL H = HIGH"]
    if isinstance(val, str):
        if "F" in val:
            df.loc[i, 'FLAT ARCH'] = True
        if "N" in val:
            df.loc[i, 'NEUTRAL ARCH'] = True
        if "H" in val:
            df.loc[i, 'HIGH ARCH'] = True
df = df.drop(columns=["ARCH TYPES F = FLAT N = NEUTRAL H = HIGH"])



In [17]:
df['FITS ORTHOTICS'] = df['FITS ORTHOTICS'].apply(lambda x: True if x == 'Y' else False)

In [18]:
df.dtypes

NAME                                                                 object
30 SEC HEATING (F) (INTERNAL)                                       float64
BOUNCE HEIGHT AVERAGE (cm)                                          float64
HEEL HEIGHT (cm)                                                    float64
HEEL-BALL DROP (cm)                                                 float64
WEIGHT oz (US MEN'S 11)                                             float64
DEGREE OF SLIP                                                      float64
SHANK SCORE                                                         float64
SPEED RATIO                                                         float64
DUROMETER                                                            object
LAST SHAPE (DEGREE OF INFLARE)                                       object
LENGTH (cm)                                                         float64
METATARSAL WIDTH(cm)                                                float64
TOEBOX WIDTH

In [19]:
df['BEST SURFACE'].unique()

array(['HARDWOOD ', 'HARDWOOD', 'ALL ', 'RUBBERIZED ', nan, 'ASPHALT',
       'ALL', 'PVC', 'HARDWOOD/ASPHALT', 'ASHALT', 'RUBBERIZED',
       'ASPHALT '], dtype=object)

In [20]:
df['BEST SURFACE'] = df['BEST SURFACE'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [21]:
df.loc[df['BEST SURFACE'] == 'ASHALT', 'BEST SURFACE'] = 'ASPHALT'

In [22]:
df['HARDWOOD SURFACE'] = False
df['RUBBERIZED SURFACE'] = False
df['ASPHALT SURFACE'] = False
for i in range(len(df)):
    val = df.loc[i, "BEST SURFACE"]
    if isinstance(val, str):
        if "HARDWOOD" in val:
            df.loc[i, 'HARDWOOD SURFACE'] = True
        if "RUBBERIZED" in val or "PVC" in val:
            df.loc[i, 'RUBBERIZED SURFACE'] = True
        if "ASPHALT" in val:
            df.loc[i, 'ASPHALT SURFACE'] = True
        
        if "ALL" in val:
            df.loc[i, 'HARDWOOD SURFACE'] = True
            df.loc[i, 'RUBBERIZED SURFACE'] = True
            df.loc[i, 'ASPHALT SURFACE'] = True

df = df.drop(columns=["BEST SURFACE"])

In [23]:
df.dtypes

NAME                                                                 object
30 SEC HEATING (F) (INTERNAL)                                       float64
BOUNCE HEIGHT AVERAGE (cm)                                          float64
HEEL HEIGHT (cm)                                                    float64
HEEL-BALL DROP (cm)                                                 float64
WEIGHT oz (US MEN'S 11)                                             float64
DEGREE OF SLIP                                                      float64
SHANK SCORE                                                         float64
SPEED RATIO                                                         float64
DUROMETER                                                            object
LAST SHAPE (DEGREE OF INFLARE)                                       object
LENGTH (cm)                                                         float64
METATARSAL WIDTH(cm)                                                float64
TOEBOX WIDTH

In [24]:
df = df.rename(columns = {'SA': 'SHOCK ABSORPTION', 'URS': 'OVERALL RATING SCORE'})

In [25]:
df['FOREFOOT HEIGHT (cm)'] = df['HEEL HEIGHT (cm)'] - df['HEEL-BALL DROP (cm)']

In [26]:
df.to_csv("data/2025 HOOP SHEET PROCESSED.csv", index=False)